In [399]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


'c:\users\lee'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


In [404]:
import numpy as np
import scipy
import pandas as pd
import math
import random
import sklearn
from sklearn.model_selection import train_test_split
from scipy.sparse.linalg import svds
import matplotlib.pyplot as plt

In [405]:
users = (pd.read_csv('C:/Users/Lee Dong woo/Documents/archive/users_interactiontea.csv'))


In [406]:
users.head(5)

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry
0,1465413032,VIEW,a18,-8.845300e+18,1.264200e+18,NaN,NaN,NaN
1,1465412560,VIEW,a41,-1.032020e+18,3.621740e+18,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US
2,1465416190,VIEW,a34,-1.130270e+18,2.631860e+18,NaN,NaN,NaN
3,1465413895,FOLLOW,a25,3.442810e+17,-3.167640e+18,NaN,NaN,NaN
4,1465412290,VIEW,a63,-4.453370e+17,5.611480e+18,NaN,NaN,NaN


In [407]:
event_type_strength ={
    'VIEW':1.0, 'LIKE' : 2.0, 'BOOKMARK': 2.5, 'FOLLOW' : 3.0, 'COMMENT CREATED' : 4.0   ##뷰(클릭수),좋아요,즐찾,팔로우,댓글 가중치 설정
}
users['eventStrength'] = (users.loc[:,'eventType'].apply (lambda d: event_type_strength[d]))
#interaction_df에서  eventType열을 불러와서 가중치 딕셔너리에서 뽑아서 eventStrength열을 생성후 대입.

In [408]:
users_over5 = (users
  .groupby('personId', group_keys=False) ##personId로 groupby
  .apply(lambda df: df.assign(interactCnt = lambda d: d['contentId'].nunique()))
  .loc[lambda d: d['interactCnt'] >= 5] ##상호작용, 즉 event가 발생한 것이 5회 이상인 행(유저) 만 불러옴
)


In [409]:
users_over5.loc[:,users_over5.columns == 'personId'].value_counts()

personId     
-1.032020e+18    1885
-1.443640e+18    1616
 3.609190e+18    1435
-2.626630e+18    1084
-3.596630e+18     903
                 ... 
-2.116520e+18       5
 6.104710e+18       5
-4.689660e+18       5
-6.361190e+18       5
-8.984150e+18       5
Length: 1298, dtype: int64

In [410]:
users_over5.loc[:,users_over5.columns == 'contentId'].value_counts()

contentId
a46          1088
a65          1065
a61          1064
a62          1059
a14          1056
             ... 
a7            962
a67           959
a19           959
a36           938
a71             4
Length: 71, dtype: int64

In [411]:
users_full_df = (
  users_over5
    .groupby(['personId', 'contentId'], as_index=False)['eventStrength'] ##한 사람이 각 콘텐츠에 행한 여러 활동들'코멘트, 클릭 등등 eventStrength의 합을 구해야함
    .sum() 
    .assign(eventScore = lambda d: np.log2(1+d['eventStrength'])) ##assign 함수로 eventScore 열 새로 할당<eventstrength에 1 더해서. 0이면 정의 안되므로 1을 더해서 보정
)##로그화 해 주는 이유는 분포를 평활하게 해 줌으로써 향후 향할 통계적 기법에 있어서 원활하게 진행하기 위함, 정규화 진행

users_full_df.head(20)

,personId,contentId,eventStrength,eventScore
0,-9.223120e+18,a1,1.0,1.000000
1,-9.223120e+18,a10,1.0,1.000000
2,-9.223120e+18,a11,1.0,1.000000
3,-9.223120e+18,a12,1.0,1.000000
4,-9.223120e+18,a13,1.0,1.000000
5,-9.223120e+18,a14,2.0,1.584963
6,-9.223120e+18,a16,1.0,1.000000
7,-9.223120e+18,a17,1.0,1.000000
8,-9.223120e+18,a18,1.0,1.000000
9,-9.223120e+18,a2,2.0,1.584963


In [239]:
from nltk.corpus import stopwords

In [412]:
tea= (pd.read_csv('C:/Users/Lee Dong woo/Documents/archive/teaable.csv'))
tea

,차품목,효능번호,카페인,향기준,품목,"맛,향"
0,Bengal Spice,1,X,2,a1,"계피, 생강"
1,BerryBlossom White,1,O,1,a2,과일향(베리향)
2,Black Cherry Berry,3,X,1,a3,"과일향(체리, 베리)"
3,Breakfast in Paris Black Tea,1,O,"1,4",a4,"라벤더꽃향, 시트러스향"
4,ButterScotch Blondie,3,O,4,a5,"버터카라멜향, 바닐라향"
...,...,...,...,...,...,...
65,요기티 Yogi Tea 스토맥 이즈 티 16티백,4,X,"2,3",a66,"감초, 카다몬씨앗, 회향씨, 페퍼민트잎, 생강뿌리, 후추"
66,요기티 Yogi Tea 진저 티 16티백,"2,4",NaN,2,a67,허브 약초 생강
67,요기티 Yogi Tea 콜드 시즌 티 16티백,4,X,"2,3,4",a68,"생강뿌리, 감초, 유칼립투스잎, 오렌지껍질, 박하잎, 레몬그라스, 바질잎, 카아몬씨..."
68,요기티 Yogi Tea 트롯 컴포트 티 허니 레몬 16티백,4,X,1,a69,"누릅나무껍질, 꿀 레몬"


In [413]:
tea.loc[69,:]

차품목     요기티 Yogi Tea 피치 디톡스 티 16티백
효능번호                             4
카페인                              X
향기준                              1
품목                             a70
맛,향                          복숭아 향
Name: 69, dtype: object

In [414]:
tea

,차품목,효능번호,카페인,향기준,품목,"맛,향"
0,Bengal Spice,1,X,2,a1,"계피, 생강"
1,BerryBlossom White,1,O,1,a2,과일향(베리향)
2,Black Cherry Berry,3,X,1,a3,"과일향(체리, 베리)"
3,Breakfast in Paris Black Tea,1,O,"1,4",a4,"라벤더꽃향, 시트러스향"
4,ButterScotch Blondie,3,O,4,a5,"버터카라멜향, 바닐라향"
...,...,...,...,...,...,...
65,요기티 Yogi Tea 스토맥 이즈 티 16티백,4,X,"2,3",a66,"감초, 카다몬씨앗, 회향씨, 페퍼민트잎, 생강뿌리, 후추"
66,요기티 Yogi Tea 진저 티 16티백,"2,4",NaN,2,a67,허브 약초 생강
67,요기티 Yogi Tea 콜드 시즌 티 16티백,4,X,"2,3,4",a68,"생강뿌리, 감초, 유칼립투스잎, 오렌지껍질, 박하잎, 레몬그라스, 바질잎, 카아몬씨..."
68,요기티 Yogi Tea 트롯 컴포트 티 허니 레몬 16티백,4,X,1,a69,"누릅나무껍질, 꿀 레몬"


In [394]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Lee Dong
[nltk_data]     woo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [395]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

In [415]:
users

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry,eventStrength
0,1465413032,VIEW,a18,-8.845300e+18,1.264200e+18,NaN,NaN,NaN,1.0
1,1465412560,VIEW,a41,-1.032020e+18,3.621740e+18,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US,1.0
2,1465416190,VIEW,a34,-1.130270e+18,2.631860e+18,NaN,NaN,NaN,1.0
3,1465413895,FOLLOW,a25,3.442810e+17,-3.167640e+18,NaN,NaN,NaN,3.0
4,1465412290,VIEW,a63,-4.453370e+17,5.611480e+18,NaN,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...
72307,1485190425,LIKE,a56,-9.016530e+18,8.614470e+18,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4...,MG,BR,2.0
72308,1485190425,VIEW,a70,1.023060e+17,5.527770e+18,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.0
72309,1485190072,VIEW,a21,-9.196670e+18,-8.300600e+18,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,SP,BR,1.0
72310,1485190434,VIEW,a62,-9.016530e+18,8.614470e+18,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4...,MG,BR,1.0


In [417]:
scent=tea.loc[:,'맛,향']
scent

0                                                계피, 생강
1                                              과일향(베리향)
2                                           과일향(체리, 베리)
3                                          라벤더꽃향, 시트러스향
4                                          버터카라멜향, 바닐라향
                            ...                        
65                      감초, 카다몬씨앗, 회향씨, 페퍼민트잎, 생강뿌리, 후추
66                                             허브 약초 생강
67    생강뿌리, 감초, 유칼립투스잎, 오렌지껍질, 박하잎, 레몬그라스, 바질잎, 카아몬씨...
68                                         누릅나무껍질, 꿀 레몬
69                                                복숭아 향
Name: 맛,향, Length: 70, dtype: object

In [418]:
tea_df=pd.DataFrame(tea)
tea_df

,차품목,효능번호,카페인,향기준,품목,"맛,향"
0,Bengal Spice,1,X,2,a1,"계피, 생강"
1,BerryBlossom White,1,O,1,a2,과일향(베리향)
2,Black Cherry Berry,3,X,1,a3,"과일향(체리, 베리)"
3,Breakfast in Paris Black Tea,1,O,"1,4",a4,"라벤더꽃향, 시트러스향"
4,ButterScotch Blondie,3,O,4,a5,"버터카라멜향, 바닐라향"
...,...,...,...,...,...,...
65,요기티 Yogi Tea 스토맥 이즈 티 16티백,4,X,"2,3",a66,"감초, 카다몬씨앗, 회향씨, 페퍼민트잎, 생강뿌리, 후추"
66,요기티 Yogi Tea 진저 티 16티백,"2,4",NaN,2,a67,허브 약초 생강
67,요기티 Yogi Tea 콜드 시즌 티 16티백,4,X,"2,3,4",a68,"생강뿌리, 감초, 유칼립투스잎, 오렌지껍질, 박하잎, 레몬그라스, 바질잎, 카아몬씨..."
68,요기티 Yogi Tea 트롯 컴포트 티 허니 레몬 16티백,4,X,1,a69,"누릅나무껍질, 꿀 레몬"


In [420]:
tea_df['scent_new']=tea_df['맛,향'].apply(remove_stopwords)
tea_df

,차품목,효능번호,카페인,향기준,품목,"맛,향",scent_new
0,Bengal Spice,1,X,2,a1,"계피, 생강","계피, 생강"
1,BerryBlossom White,1,O,1,a2,과일향(베리향),과일향(베리향)
2,Black Cherry Berry,3,X,1,a3,"과일향(체리, 베리)","과일향(체리, 베리)"
3,Breakfast in Paris Black Tea,1,O,"1,4",a4,"라벤더꽃향, 시트러스향","라벤더꽃향, 시트러스향"
4,ButterScotch Blondie,3,O,4,a5,"버터카라멜향, 바닐라향","버터카라멜향, 바닐라향"
...,...,...,...,...,...,...,...
65,요기티 Yogi Tea 스토맥 이즈 티 16티백,4,X,"2,3",a66,"감초, 카다몬씨앗, 회향씨, 페퍼민트잎, 생강뿌리, 후추","감초, 카다몬씨앗, 회향씨, 페퍼민트잎, 생강뿌리, 후추"
66,요기티 Yogi Tea 진저 티 16티백,"2,4",NaN,2,a67,허브 약초 생강,허브 약초 생강
67,요기티 Yogi Tea 콜드 시즌 티 16티백,4,X,"2,3,4",a68,"생강뿌리, 감초, 유칼립투스잎, 오렌지껍질, 박하잎, 레몬그라스, 바질잎, 카아몬씨...","생강뿌리, 감초, 유칼립투스잎, 오렌지껍질, 박하잎, 레몬그라스, 바질잎, 카아몬씨..."
68,요기티 Yogi Tea 트롯 컴포트 티 허니 레몬 16티백,4,X,1,a69,"누릅나무껍질, 꿀 레몬","누릅나무껍질, 꿀 레몬"


In [426]:
#한국어 불용어 없애기
import string
stopwords_to_remove=[',','향','(',')']
tea_df.reset_index(drop=True, inplace=True)
# Define our function to remove stopwords
def remove_stopwords(word):
    if word not in stopwords_to_remove:
        return word
    else:
        return ''
# Apply the function to our words column to clean up.
tea_df['words_clean'] = tea_df['scent_new'].apply(remove_stopwords)
tea_df

,차품목,효능번호,카페인,향기준,품목,"맛,향",scent_new,words_clean
0,Bengal Spice,1,X,2,a1,"계피, 생강","계피, 생강","계피, 생강"
1,BerryBlossom White,1,O,1,a2,과일향(베리향),과일향(베리향),과일향(베리향)
2,Black Cherry Berry,3,X,1,a3,"과일향(체리, 베리)","과일향(체리, 베리)","과일향(체리, 베리)"
3,Breakfast in Paris Black Tea,1,O,"1,4",a4,"라벤더꽃향, 시트러스향","라벤더꽃향, 시트러스향","라벤더꽃향, 시트러스향"
4,ButterScotch Blondie,3,O,4,a5,"버터카라멜향, 바닐라향","버터카라멜향, 바닐라향","버터카라멜향, 바닐라향"
...,...,...,...,...,...,...,...,...
65,요기티 Yogi Tea 스토맥 이즈 티 16티백,4,X,"2,3",a66,"감초, 카다몬씨앗, 회향씨, 페퍼민트잎, 생강뿌리, 후추","감초, 카다몬씨앗, 회향씨, 페퍼민트잎, 생강뿌리, 후추","감초, 카다몬씨앗, 회향씨, 페퍼민트잎, 생강뿌리, 후추"
66,요기티 Yogi Tea 진저 티 16티백,"2,4",NaN,2,a67,허브 약초 생강,허브 약초 생강,허브 약초 생강
67,요기티 Yogi Tea 콜드 시즌 티 16티백,4,X,"2,3,4",a68,"생강뿌리, 감초, 유칼립투스잎, 오렌지껍질, 박하잎, 레몬그라스, 바질잎, 카아몬씨...","생강뿌리, 감초, 유칼립투스잎, 오렌지껍질, 박하잎, 레몬그라스, 바질잎, 카아몬씨...","생강뿌리, 감초, 유칼립투스잎, 오렌지껍질, 박하잎, 레몬그라스, 바질잎, 카아몬씨..."
68,요기티 Yogi Tea 트롯 컴포트 티 허니 레몬 16티백,4,X,1,a69,"누릅나무껍질, 꿀 레몬","누릅나무껍질, 꿀 레몬","누릅나무껍질, 꿀 레몬"


In [427]:
tea_new = (pd.read_csv('C:/Users/Lee Dong woo/Documents/archive/teaable2.csv'))
tea_new

,차품목,효능번호,카페인,향기준,품목,"맛,향"
0,Bengal Spice,1,X,2,a1,"계피 향, 생강 향"
1,BerryBlossom White,1,O,1,a2,베리 향
2,Black Cherry Berry,3,X,1,a3,"체리 향, 베리 향"
3,Breakfast in Paris Black Tea,1,O,"1,4",a4,"라벤더꽃 향, 시트러스 향"
4,ButterScotch Blondie,3,O,4,a5,"버터카라멜 향, 바닐라 향"
...,...,...,...,...,...,...
65,요기티 Yogi Tea 스토맥 이즈 티 16티백,4,X,"2,3",a66,"감초 향, 카다몬 향, 회향씨 향, 박하 향, 생강뿌리 향, 후추 향"
66,요기티 Yogi Tea 진저 티 16티백,"2,4",NaN,2,a67,허브 약초 생강
67,요기티 Yogi Tea 콜드 시즌 티 16티백,4,X,"2,3,4",a68,"생강뿌리 향, 감초 향, 유칼립투스잎 향, 오렌지껍질 향, 박하 향, 레몬그라스 향..."
68,요기티 Yogi Tea 트롯 컴포트 티 허니 레몬 16티백,4,X,1,a69,"누릅나무껍질 향, 꿀 향, 레몬 향"


In [428]:
tea_new=pd.DataFrame(tea_new)
tea_new['scent_new']=tea_new['맛,향'].apply(remove_stopwords)
tea_new.head(15)

,차품목,효능번호,카페인,향기준,품목,"맛,향",scent_new
0,Bengal Spice,1,X,2,a1,"계피 향, 생강 향","계피 향, 생강 향"
1,BerryBlossom White,1,O,1,a2,베리 향,베리 향
2,Black Cherry Berry,3,X,1,a3,"체리 향, 베리 향","체리 향, 베리 향"
3,Breakfast in Paris Black Tea,1,O,"1,4",a4,"라벤더꽃 향, 시트러스 향","라벤더꽃 향, 시트러스 향"
4,ButterScotch Blondie,3,O,4,a5,"버터카라멜 향, 바닐라 향","버터카라멜 향, 바닐라 향"
5,Candy Cane Lane,4,△,"3,5",a6,"민트 향, 녹차 향","민트 향, 녹차 향"
6,Chai Green Tea,3,O,"2,5",a7,"계피 향, 생강 향, 녹차 향","계피 향, 생강 향, 녹차 향"
7,Chamomile Nights Herbal Tea,4,X,4,a8,캐모마일 향,캐모마일 향
8,China Green Tips,3,O,5,a9,녹차 향,녹차 향
9,Choco,3,X,2,a10,"초콜릿 향, 계피 향, 생강 향","초콜릿 향, 계피 향, 생강 향"


In [439]:
tea_new['scent_new'] = tea_new.scent_new.str.replace('향,?' , '')
tea_new


<ipython-input-439-c912b31a11a1>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  tea_new['scent_new'] = tea_new.scent_new.str.replace('향,?' , '')


,차품목,효능번호,카페인,향기준,품목,"맛,향",scent_new
0,Bengal Spice,1,X,2,a1,"계피 향, 생강 향",계피 생강
1,BerryBlossom White,1,O,1,a2,베리 향,베리
2,Black Cherry Berry,3,X,1,a3,"체리 향, 베리 향",체리 베리
3,Breakfast in Paris Black Tea,1,O,"1,4",a4,"라벤더꽃 향, 시트러스 향",라벤더꽃 시트러스
4,ButterScotch Blondie,3,O,4,a5,"버터카라멜 향, 바닐라 향",버터카라멜 바닐라
...,...,...,...,...,...,...,...
65,요기티 Yogi Tea 스토맥 이즈 티 16티백,4,X,"2,3",a66,"감초 향, 카다몬 향, 회향씨 향, 박하 향, 생강뿌리 향, 후추 향",감초 카다몬 회씨 박하 생강뿌리 후추
66,요기티 Yogi Tea 진저 티 16티백,"2,4",NaN,2,a67,허브 약초 생강,허브 약초 생강
67,요기티 Yogi Tea 콜드 시즌 티 16티백,4,X,"2,3,4",a68,"생강뿌리 향, 감초 향, 유칼립투스잎 향, 오렌지껍질 향, 박하 향, 레몬그라스 향...",생강뿌리 감초 유칼립투스잎 오렌지껍질 박하 레몬그라스 바질 카다몬 오...
68,요기티 Yogi Tea 트롯 컴포트 티 허니 레몬 16티백,4,X,1,a69,"누릅나무껍질 향, 꿀 향, 레몬 향",누릅나무껍질 꿀 레몬


In [430]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to C:\Users\Lee Dong
[nltk_data]     woo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Lee Dong woo\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [440]:
def extract_noun(words):
    token = nltk.tokenize.word_tokenize(words)
    result=[]
    for i in nltk.pos_tag(token):
        if i[1].startswith('NN'):
            result.append(i[0])

    return(', '.join(result))
tea_new['scent_words']=tea_new['scent_new'].apply(extract_noun)
tea_new

,차품목,효능번호,카페인,향기준,품목,"맛,향",scent_new,scent_words
0,Bengal Spice,1,X,2,a1,"계피 향, 생강 향",계피 생강,"계피, 생강"
1,BerryBlossom White,1,O,1,a2,베리 향,베리,베리
2,Black Cherry Berry,3,X,1,a3,"체리 향, 베리 향",체리 베리,"체리, 베리"
3,Breakfast in Paris Black Tea,1,O,"1,4",a4,"라벤더꽃 향, 시트러스 향",라벤더꽃 시트러스,"라벤더꽃, 시트러스"
4,ButterScotch Blondie,3,O,4,a5,"버터카라멜 향, 바닐라 향",버터카라멜 바닐라,"버터카라멜, 바닐라"
...,...,...,...,...,...,...,...,...
65,요기티 Yogi Tea 스토맥 이즈 티 16티백,4,X,"2,3",a66,"감초 향, 카다몬 향, 회향씨 향, 박하 향, 생강뿌리 향, 후추 향",감초 카다몬 회씨 박하 생강뿌리 후추,"카다몬, 회씨, 박하, 생강뿌리, 후추"
66,요기티 Yogi Tea 진저 티 16티백,"2,4",NaN,2,a67,허브 약초 생강,허브 약초 생강,"약초, 생강"
67,요기티 Yogi Tea 콜드 시즌 티 16티백,4,X,"2,3,4",a68,"생강뿌리 향, 감초 향, 유칼립투스잎 향, 오렌지껍질 향, 박하 향, 레몬그라스 향...",생강뿌리 감초 유칼립투스잎 오렌지껍질 박하 레몬그라스 바질 카다몬 오...,"감초, 유칼립투스잎, 오렌지껍질, 박하, 레몬그라스, 바질, 카다몬, 오레가노, 파슬리"
68,요기티 Yogi Tea 트롯 컴포트 티 허니 레몬 16티백,4,X,1,a69,"누릅나무껍질 향, 꿀 향, 레몬 향",누릅나무껍질 꿀 레몬,"꿀, 레몬"


In [441]:
tea_new.drop('맛,향',axis=1)

,차품목,효능번호,카페인,향기준,품목,scent_new,scent_words
0,Bengal Spice,1,X,2,a1,계피 생강,"계피, 생강"
1,BerryBlossom White,1,O,1,a2,베리,베리
2,Black Cherry Berry,3,X,1,a3,체리 베리,"체리, 베리"
3,Breakfast in Paris Black Tea,1,O,"1,4",a4,라벤더꽃 시트러스,"라벤더꽃, 시트러스"
4,ButterScotch Blondie,3,O,4,a5,버터카라멜 바닐라,"버터카라멜, 바닐라"
...,...,...,...,...,...,...,...
65,요기티 Yogi Tea 스토맥 이즈 티 16티백,4,X,"2,3",a66,감초 카다몬 회씨 박하 생강뿌리 후추,"카다몬, 회씨, 박하, 생강뿌리, 후추"
66,요기티 Yogi Tea 진저 티 16티백,"2,4",NaN,2,a67,허브 약초 생강,"약초, 생강"
67,요기티 Yogi Tea 콜드 시즌 티 16티백,4,X,"2,3,4",a68,생강뿌리 감초 유칼립투스잎 오렌지껍질 박하 레몬그라스 바질 카다몬 오...,"감초, 유칼립투스잎, 오렌지껍질, 박하, 레몬그라스, 바질, 카다몬, 오레가노, 파슬리"
68,요기티 Yogi Tea 트롯 컴포트 티 허니 레몬 16티백,4,X,1,a69,누릅나무껍질 꿀 레몬,"꿀, 레몬"


In [442]:
tea_new.drop(['scent_words','맛,향','향기준'],axis=1,inplace=True)

In [443]:
tea_new

,차품목,효능번호,카페인,품목,scent_new
0,Bengal Spice,1,X,a1,계피 생강
1,BerryBlossom White,1,O,a2,베리
2,Black Cherry Berry,3,X,a3,체리 베리
3,Breakfast in Paris Black Tea,1,O,a4,라벤더꽃 시트러스
4,ButterScotch Blondie,3,O,a5,버터카라멜 바닐라
...,...,...,...,...,...
65,요기티 Yogi Tea 스토맥 이즈 티 16티백,4,X,a66,감초 카다몬 회씨 박하 생강뿌리 후추
66,요기티 Yogi Tea 진저 티 16티백,"2,4",NaN,a67,허브 약초 생강
67,요기티 Yogi Tea 콜드 시즌 티 16티백,4,X,a68,생강뿌리 감초 유칼립투스잎 오렌지껍질 박하 레몬그라스 바질 카다몬 오...
68,요기티 Yogi Tea 트롯 컴포트 티 허니 레몬 16티백,4,X,a69,누릅나무껍질 꿀 레몬


In [444]:
tea_new=tea_new.fillna('X')

In [438]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [336]:
vectorizer = TfidfVectorizer(
    analyzer='word',
    ngram_range=(1, 2),
    min_df=0.003,
    max_df=0.5,
    max_features=5000,
    stop_words=kor_stopwords
)

In [479]:
item_ids = tea_new['품목'].tolist()
tfidf_matrix = vectorizer.fit_transform(tea_new['scent_new'])
tfidf_feature_names = vectorizer.get_feature_names()

In [480]:
tfidf_matrix

<70x131 sparse matrix of type '<class 'numpy.float64'>'
	with 222 stored elements in Compressed Sparse Row format>

In [481]:
users

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry,eventStrength
0,1465413032,VIEW,a18,-8.845300e+18,1.264200e+18,NaN,NaN,NaN,1.0
1,1465412560,VIEW,a41,-1.032020e+18,3.621740e+18,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US,1.0
2,1465416190,VIEW,a34,-1.130270e+18,2.631860e+18,NaN,NaN,NaN,1.0
3,1465413895,FOLLOW,a25,3.442810e+17,-3.167640e+18,NaN,NaN,NaN,3.0
4,1465412290,VIEW,a63,-4.453370e+17,5.611480e+18,NaN,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...
72307,1485190425,LIKE,a56,-9.016530e+18,8.614470e+18,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4...,MG,BR,2.0
72308,1485190425,VIEW,a70,1.023060e+17,5.527770e+18,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.0
72309,1485190072,VIEW,a21,-9.196670e+18,-8.300600e+18,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,SP,BR,1.0
72310,1485190434,VIEW,a62,-9.016530e+18,8.614470e+18,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4...,MG,BR,1.0


In [482]:
def get_item_profile(item_id):
    idx = item_ids.index(item_id)
    item_profile = tfidf_matrix[idx:idx+1]
    return item_profile

def get_item_profiles(ids):
    item_profiles_list = [get_item_profile(x) for x in ids]
    item_profiles = scipy.sparse.vstack(item_profiles_list)
    return item_profiles

def build_user_profile(person_id, interaction_indexed_df):
    interaction_person_df = interaction_indexed_df.loc[person_id]
    user_item_profiles = get_item_profiles(interaction_person_df['contentId'])
    
    user_item_strengths = np.array(interaction_person_df['eventStrength']).reshape(-1, 1)
    
    # 상호작용 강도를 바탕으로 가중치를 부여하여 평균 계산한다
    user_item_strengths_weighted_avg = \
        np.sum(user_item_profiles.multiply(user_item_strengths), axis=0) /\
        np.sum(user_item_strengths)
        
    user_profile_norm = sklearn.preprocessing.normalize(user_item_strengths_weighted_avg)
    return user_profile_norm

def build_user_profiles():
    interaction_indexed_df = (users_full_df
        .loc[lambda d: d['contentId'].isin(tea_new['품목'])]
        .set_index('personId')
    )
    user_profiles = {}
    
    for person_id in interaction_indexed_df.index.unique():
        user_profiles[person_id] = build_user_profile(person_id, interaction_indexed_df)
        
    return user_profiles

In [483]:
user_profiles = build_user_profiles()
len(user_profiles)

1298

In [484]:
myprofile = user_profiles[-8.845300e+18].flatten().tolist()
pd.DataFrame(sorted(zip(tfidf_feature_names, myprofile), key=lambda x: -x[1])[:20],
             columns=['token', 'relevance'])

,token,relevance
0,홍차,0.405555
1,레몬,0.305867
2,계피,0.293134
3,생강,0.263664
4,민트,0.258612
5,과일,0.212008
6,계피 생강,0.198672
7,베리,0.195850
8,마테,0.184772
9,라즈베리,0.176948
